In [2]:
import pandas as pd
import glob
glob.glob("*.gz")

['GSE17308_series_matrix.txt.gz', 'GSE9891_series_matrix.txt.gz']

In [7]:
df = pd.read_csv("GSE9891_series_matrix.txt.gz",sep="\t",comment="!",index_col=0)

In [8]:
df.head()

,GSM249714,GSM249715,GSM249716,GSM249717,GSM249718,GSM249719,GSM249720,GSM249721,GSM249722,GSM249723,...,GSM249992,GSM249993,GSM249994,GSM249995,GSM249996,GSM249997,GSM249998,GSM249999,GSM250000,GSM250001
ID_REF,,,,,,,,,,,,,,,,,,,,,
1007_s_at,10.03755,10.59104,10.29167,9.96189,10.81203,10.82938,10.35202,10.53426,10.59050,10.04427,...,10.09685,10.43477,10.49728,9.83905,10.46557,10.20668,10.29898,9.74905,8.94333,9.57596
1053_at,6.79370,7.70975,6.64165,6.59782,6.95447,6.49843,6.92777,6.95893,6.79814,6.95434,...,6.71268,7.36401,6.71602,6.85716,7.15526,7.17858,7.18437,7.93743,8.31758,7.36348
117_at,5.75819,5.75097,5.87125,5.68974,6.27595,6.30827,6.61096,6.88128,6.11965,5.27755,...,6.53972,6.27487,6.86924,5.32604,5.33534,5.97113,5.70538,5.35259,6.25604,6.39594
121_at,10.39472,10.45093,10.43725,9.97365,10.15632,10.54951,9.77316,10.03439,10.79133,9.56140,...,9.64454,9.75665,9.12342,10.40375,9.82070,9.77309,10.17110,11.00313,11.23231,9.18911
1255_g_at,2.94816,3.11233,3.05501,3.02842,2.96967,3.17534,3.14297,2.98310,2.98802,3.00182,...,3.16375,3.01850,2.94468,3.12271,2.97030,2.83417,3.22609,2.96651,3.11244,2.98776


In [9]:
df.max().max()

14.814279999999998

In [10]:
# to table LMP vs PSO

In [50]:
import numpy as np
from scipy import stats
selected_genes = ["SLC7A2","BBS12","RPL7A","HES2","RPS12","RPL12","PIFO","RPS15","AFF2","CRABP2","MAFB"]
# selected_genes = ["CHST15","TSPAN1","PPP1R14C","ECE1","CLIC1","CA12","TMEM30B","ENPP4","ANK1","SYNPO","GCLC","NQO1","PGK1","PDGFC","SERPINE2","ZDHHC7","DNAJB9"]
out = []
for gene in selected_genes:
	lines = open(gene+".data").readlines()[1:]
	my_dict  = {}
	for line in lines:
		line = line.strip().split()
		key = line[0]
		status = line[3]
		if key in my_dict:
			my_dict[key][status].append(float(line[1]))
		else:
			my_dict[key] = {}
			my_dict[key]["LMP"] = []
			my_dict[key]["Malignant"] = []
			my_dict[key][status].append(float(line[1]))
	for key in my_dict:
		LMP_median = np.mean(my_dict[key]["LMP"])
		Malignant_median = np.mean(my_dict[key]["Malignant"])
		LMP_std = np.std(my_dict[key]["LMP"])
		Malignant_std = np.std(my_dict[key]["Malignant"])
		LMP_mean = np.mean(my_dict[key]["LMP"])
		Malignant_mean = np.mean(my_dict[key]["Malignant"])
		fc = LMP_mean-Malignant_mean
		p_value = stats.ttest_ind(my_dict[key]["LMP"], my_dict[key]["Malignant"], equal_var = False)[1]
		out.append([gene,key,LMP_median,LMP_std,Malignant_median,Malignant_std,fc,p_value])		

df2 = pd.DataFrame(out)

print (df2.head())
df2[2] = df2[2].apply(lambda x:2**x)
df2[6] = df2[6].apply(lambda x:(2**abs(x))*np.sign(-x))
df2 = df2[df2[7]<=0.01]
df2 = df2.sort_values(6)
df2.columns = ['Gene','Probe','Mean(SBT)','Std Dev(SBT)','Mean(HGSOC)','Std Dev(HGSOC)','Fold Change','P-value']




        0            1          2         3          4         5         6  \
0  SLC7A2  207626_s_at   3.451144  0.304541   3.324969  0.329356  0.126175   
1  SLC7A2    225516_at  10.914176  0.840472   7.444828  1.555641  3.469348   
2   BBS12    229603_at   6.750868  0.427343   5.256885  0.781521  1.493983   
3   RPL7A  217740_x_at  13.923346  0.079070  13.559442  0.301818  0.363904   
4   RPL7A  224930_x_at  13.784522  0.122066  13.351512  0.352326  0.433010   

              7  
0  1.153120e-01  
1  2.468924e-14  
2  1.120945e-12  
3  4.480749e-20  
4  1.712228e-14  


In [52]:
df2.to_csv("Table_S5.csv",index=False)

In [46]:
np.sign(0.2)

1.0

In [41]:
df2.shape

(20, 8)

In [ ]:
df2['']

In [26]:
df2 = df2.sort_values(7,ascending=False)
df2.head(n=10)

,0,1,2,3,4,5,6,7
14,AFF2,216364_s_at,4.100413,0.200394,4.109849,0.265069,-0.009437,8.556744e-01
15,AFF2,210957_s_at,3.911968,0.255264,3.884236,0.218534,0.027732,6.665643e-01
0,SLC7A2,207626_s_at,3.451144,0.304541,3.324969,0.329356,0.126175,1.153120e-01
5,HES2,214521_at,3.946308,0.274520,3.802730,0.259323,0.143579,4.956870e-02
6,HES2,216674_at,3.671042,0.267081,3.529165,0.221545,0.141877,4.556477e-02
16,AFF2,206105_at,4.290967,0.525364,3.986291,0.352705,0.304676,2.992983e-02
7,HES2,231928_at,7.169783,0.596539,6.166617,0.438257,1.003166,2.088536e-06
12,PIFO,228100_at,9.386173,1.159407,6.716917,1.220685,2.669256,1.662089e-08
17,CRABP2,202575_at,7.851249,0.959819,10.389143,1.144796,-2.537894,1.362940e-09
19,MAFB,218559_s_at,7.817758,0.523164,9.589229,1.032783,-1.771471,1.355900e-12


In [27]:
df2 = df2[df2[7]<=0.01]

In [28]:
df2.shape

(14, 8)

In [29]:
df2.head()

,0,1,2,3,4,5,6,7
7,HES2,231928_at,7.169783,0.596539,6.166617,0.438257,1.003166,2.088536e-06
12,PIFO,228100_at,9.386173,1.159407,6.716917,1.220685,2.669256,1.662089e-08
17,CRABP2,202575_at,7.851249,0.959819,10.389143,1.144796,-2.537894,1.362940e-09
19,MAFB,218559_s_at,7.817758,0.523164,9.589229,1.032783,-1.771471,1.355900e-12
2,BBS12,229603_at,6.750868,0.427343,5.256885,0.781521,1.493983,1.120945e-12


In [30]:
df2 = df2.sort_values(6)
df2.head()

,0,1,2,3,4,5,6,7
17,CRABP2,202575_at,7.851249,0.959819,10.389143,1.144796,-2.537894,1.362940e-09
19,MAFB,218559_s_at,7.817758,0.523164,9.589229,1.032783,-1.771471,1.355900e-12
18,MAFB,222670_s_at,7.222535,0.380536,8.681577,0.909414,-1.459042,1.199966e-14
13,RPS15,200819_s_at,13.575302,0.078822,13.332552,0.288019,0.242750,5.546981e-13
8,RPS12,213377_x_at,13.750204,0.084611,13.477001,0.278134,0.273204,1.430933e-13


In [31]:
df2.tail()

,0,1,2,3,4,5,6,7
4,RPL7A,224930_x_at,13.784522,0.122066,13.351512,0.352326,0.433010,1.712228e-14
7,HES2,231928_at,7.169783,0.596539,6.166617,0.438257,1.003166,2.088536e-06
2,BBS12,229603_at,6.750868,0.427343,5.256885,0.781521,1.493983,1.120945e-12
12,PIFO,228100_at,9.386173,1.159407,6.716917,1.220685,2.669256,1.662089e-08
1,SLC7A2,225516_at,10.914176,0.840472,7.444828,1.555641,3.469348,2.468924e-14


In [38]:
2**0.4

1.3195079107728942

In [35]:
df2 = df2[df2[6].abs()>=1]

In [36]:
df2

,0,1,2,3,4,5,6,7
17,CRABP2,202575_at,7.851249,0.959819,10.389143,1.144796,-2.537894,1.362940e-09
19,MAFB,218559_s_at,7.817758,0.523164,9.589229,1.032783,-1.771471,1.355900e-12
18,MAFB,222670_s_at,7.222535,0.380536,8.681577,0.909414,-1.459042,1.199966e-14
7,HES2,231928_at,7.169783,0.596539,6.166617,0.438257,1.003166,2.088536e-06
2,BBS12,229603_at,6.750868,0.427343,5.256885,0.781521,1.493983,1.120945e-12
12,PIFO,228100_at,9.386173,1.159407,6.716917,1.220685,2.669256,1.662089e-08
1,SLC7A2,225516_at,10.914176,0.840472,7.444828,1.555641,3.469348,2.468924e-14
